<a href="https://colab.research.google.com/github/aekanun2020/AdvancedStat/blob/main/prove_one_neuronRNN_3datapoint_sameasPart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn.functional as F
import math
import numpy as np

# กำหนด seed เพื่อให้ผลลัพธ์คงที่
torch.manual_seed(9)

# ==================== ส่วนที่ 1: การเตรียมโมเดลและข้อมูล ====================
print("=" * 60)
print("ส่วนที่ 1: การเตรียมโมเดลและข้อมูล")
print("=" * 60)

# สร้าง RNN Cell โดยมี input_size=1, hidden_size=1, และไม่มี bias
rnn_model = torch.nn.RNNCell(input_size=1, hidden_size=1, bias=False)

# Linear layer สำหรับแปลง hidden state เป็น output
output_layer = torch.nn.Linear(1, 1, bias=False)

# แสดงค่า parameters ทั้งหมดใน RNN Cell พร้อมคำอธิบาย
weights = list(rnn_model.parameters())
w_ih = weights[0]  # Input-to-Hidden (Weight W)
w_hh = weights[1]  # Hidden-to-Hidden (Weight U)
print(f"Parameters เริ่มต้นของ RNN Cell:")
print(f"Weight W (input-to-hidden): {w_ih.item():.4f}")
print(f"Weight U (hidden-to-hidden): {w_hh.item():.4f}")

# กำหนดค่า V
with torch.no_grad():
    output_layer.weight.copy_(torch.tensor([[0.5]]))
v_weight = output_layer.weight.item()
print(f"Weight V (hidden-to-output): {v_weight:.4f}")

# สร้างข้อมูล sine wave ความยาว 20 ค่า
seq_length = 20
time_steps = torch.linspace(0, 4*math.pi, seq_length)
data = torch.sin(time_steps).unsqueeze(1)

# แทนที่ 3 ค่าแรกด้วย [1, -0.7775, 0.2588]
data[0, 0] = 1.0
data[1, 0] = -0.7775
data[2, 0] = 0.2588

# แบ่งเป็น input และ target (เราต้องการทำนาย data[t+1] จาก data[t])
x_train = data[:-1]  # ทุกค่ายกเว้นค่าสุดท้าย (0-18)
y_train = data[1:]   # ทุกค่ายกเว้นค่าแรก (1-19)

print(f"\nสร้างชุดข้อมูลผสมความยาว {seq_length} ค่า (3 ค่าแรกคือ [1, -0.7775, 0.2588] ส่วนที่เหลือเป็น sine wave)")
print(f"จำนวน time steps ในชุดข้อมูล: {len(x_train)}")
print(f"Shape ของ input (x): {x_train.shape}, target (y): {y_train.shape}")

print("\nข้อมูลทั้งหมด 20 ค่า:")
print("  ลำดับ |    ค่าข้อมูล    ")
print("-" * 25)
for i in range(len(data)):
    print(f"   {i+1:2d}   |    {data[i].item():+.6f}    ")

print("\nข้อมูล input และ target ทั้งหมด:")
print("  Time Step |    Input (x)    |   Target (y)   ")
print("-" * 50)
for i in range(len(x_train)):
    print(f"     {i+1:2d}     |    {x_train[i].item():+.6f}    |    {y_train[i].item():+.6f}    ")

# ==================== ส่วนที่ 2: การคำนวณก่อนการเทรน ====================
print("\n" + "=" * 60)
print("ส่วนที่ 2: การคำนวณก่อนการเทรน (Forward Pass แบบละเอียด)")
print("=" * 60)

# คำนวณ forward pass ก่อนการเทรน
h = torch.zeros(1, 1)  # เริ่มต้นด้วย hidden state = 0

print("\nการคำนวณแบบละเอียดในแต่ละ time step ก่อนการเทรน:")
print("-" * 90)
print("Time Step |   Input (x)  |    h_{t-1}    |       W·x       |       U·h       |     h_t = tanh(W·x + U·h)     |      y_t = sigmoid(V·h_t)     |   Target   |    Error   ")
print("-" * 90)

total_squared_error = 0
predictions = []

for t in range(len(x_train)):
    # รับค่า input และค่า target
    x_t = x_train[t].unsqueeze(0)
    y_target = y_train[t].item()

    # คำนวณ forward pass ของ RNN Cell
    Wx = w_ih.item() * x_t.item()
    Uh = w_hh.item() * h.item()
    next_h = torch.tanh(rnn_model.weight_ih * x_t + rnn_model.weight_hh * h)
    y_pred = torch.sigmoid(output_layer(next_h))

    # คำนวณ error
    error = y_pred.item() - y_target
    squared_error = error ** 2
    total_squared_error += squared_error

    # เก็บผลการทำนาย
    predictions.append(y_pred.item())

    # แสดงข้อมูลการคำนวณ
    print(f"    {t+1:2d}    | {x_t.item():+.6f} | {h.item():+.6f} | {Wx:+.6f} | {Uh:+.6f} | {next_h.item():+.6f} | {y_pred.item():+.6f} | {y_target:+.6f} | {error:+.6f}")

    # อัพเดท hidden state สำหรับ time step ถัดไป
    h = next_h

# คำนวณ MSE
mse = total_squared_error / len(x_train)
print("-" * 90)
print(f"Mean Squared Error (MSE) ก่อนการเทรน: {mse:.6f}")

# ==================== ส่วนที่ 3: การเทรนโมเดล ====================
print("\n" + "=" * 60)
print("ส่วนที่ 3: การเทรนโมเดล RNN")
print("=" * 60)

# สร้างโมเดลใหม่เพื่อการเทรน (ใช้ค่าเริ่มต้นเดียวกับโมเดลก่อนหน้า)
torch.manual_seed(9)
rnn_train = torch.nn.RNNCell(input_size=1, hidden_size=1, bias=False)
output_train = torch.nn.Linear(1, 1, bias=False)

# กำหนดค่าเริ่มต้นให้เหมือนกับโมเดลก่อนหน้า
with torch.no_grad():
    rnn_train.weight_ih.copy_(w_ih)
    rnn_train.weight_hh.copy_(w_hh)
    output_train.weight.copy_(torch.tensor([[0.5]]))

# ตรวจสอบค่าเริ่มต้น
print("Parameters ก่อนการเทรน:")
print(f"  W: {rnn_train.weight_ih.item():.6f}")
print(f"  U: {rnn_train.weight_hh.item():.6f}")
print(f"  V: {output_train.weight.item():.6f}")

# สร้าง optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(list(rnn_train.parameters()) + list(output_train.parameters()), lr=learning_rate)
print(f"\nOptimizer: SGD with learning rate = {learning_rate}")
print("Loss function: Mean Squared Error (MSE)")

# จำนวน epochs
num_epochs = 100
print(f"\nการเทรนจะใช้ {num_epochs} epochs กับข้อมูล {len(x_train)} time steps")

# เตรียมตัวแปรสำหรับเก็บประวัติ
losses = []
w_history = [rnn_train.weight_ih.item()]
u_history = [rnn_train.weight_hh.item()]
v_history = [output_train.weight.item()]

# Train โมเดล
print("\nเริ่มการเทรน...")
for epoch in range(num_epochs):
    # เริ่มต้นจาก hidden state เป็น 0
    h = torch.zeros(1, 1)
    h.requires_grad = True  # ต้องเก็บ gradient เพื่อการเทรน

    # เก็บค่าทำนายทั้งหมด
    predictions = []

    # Forward pass ผ่านทุก time steps
    for t in range(len(x_train)):
        h = rnn_train(x_train[t].unsqueeze(0), h)
        y_pred = output_train(h)
        predictions.append(y_pred)

    # รวมการทำนายทั้งหมดเป็น tensor เดียว
    predictions = torch.cat(predictions)

    # คำนวณ loss
    loss = F.mse_loss(predictions, y_train)
    losses.append(loss.item())

    # Backward pass และการปรับค่า parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # เก็บค่า parameters หลังการปรับปรุง
    w_history.append(rnn_train.weight_ih.item())
    u_history.append(rnn_train.weight_hh.item())
    v_history.append(output_train.weight.item())

    # แสดงความคืบหน้า
    if (epoch+1) % 10 == 0 or epoch == 0 or epoch == num_epochs-1:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")
        print(f"  W: {rnn_train.weight_ih.item():.6f}, U: {rnn_train.weight_hh.item():.6f}, V: {output_train.weight.item():.6f}")

# แสดงการเปลี่ยนแปลงของ parameters
print("\nการเปลี่ยนแปลงของ parameters ในแต่ละ epoch:")
print("  Epoch |      W       |      U       |      V       |    Loss    ")
print("-" * 60)
selected_epochs = [0, 9, 19, 29, 49]  # แสดงเฉพาะบาง epochs เพื่อให้กระชับ
for i in selected_epochs:
    epoch_label = "Initial" if i == 0 else f"{i+1:5d}"
    loss_value = losses[i] if i > 0 else mse
    print(f"  {epoch_label} | {w_history[i]:+.6f} | {u_history[i]:+.6f} | {v_history[i]:+.6f} | {loss_value:.6f}")

# ==================== ส่วนที่ 4: การคำนวณหลังการเทรน ====================
print("\n" + "=" * 60)
print("ส่วนที่ 4: การคำนวณหลังการเทรน (Forward Pass แบบละเอียด)")
print("=" * 60)

# คำนวณ forward pass หลังการเทรน
h = torch.zeros(1, 1)  # เริ่มต้นด้วย hidden state = 0

print("\nการคำนวณแบบละเอียดในแต่ละ time step หลังการเทรน:")
print("-" * 90)
print("Time Step |   Input (x)  |    h_{t-1}    |       W·x       |       U·h       |     h_t = tanh(W·x + U·h)     |      y_t = sigmoid(V·h_t)     |   Target   |    Error   ")
print("-" * 90)

total_squared_error = 0
final_predictions = []

with torch.no_grad():  # ไม่ต้องคำนวณ gradient ในการทดสอบ
    for t in range(len(x_train)):
        # รับค่า input และค่า target
        x_t = x_train[t].unsqueeze(0)
        y_target = y_train[t].item()

        # คำนวณ forward pass ของ RNN Cell
        Wx = rnn_train.weight_ih.item() * x_t.item()
        Uh = rnn_train.weight_hh.item() * h.item()
        next_h = torch.tanh(rnn_train.weight_ih * x_t + rnn_train.weight_hh * h)
        y_pred = torch.sigmoid(output_train(next_h))

        # คำนวณ error
        error = y_pred.item() - y_target
        squared_error = error ** 2
        total_squared_error += squared_error

        # เก็บผลการทำนาย
        final_predictions.append(y_pred.item())

        # แสดงข้อมูลการคำนวณ
        print(f"    {t+1:2d}    | {x_t.item():+.6f} | {h.item():+.6f} | {Wx:+.6f} | {Uh:+.6f} | {next_h.item():+.6f} | {y_pred.item():+.6f} | {y_target:+.6f} | {error:+.6f}")

        # อัพเดท hidden state สำหรับ time step ถัดไป
        h = next_h

# คำนวณ MSE
final_mse = total_squared_error / len(x_train)
print("-" * 90)
print(f"Mean Squared Error (MSE) หลังการเทรน: {final_mse:.6f}")
print(f"เปรียบเทียบกับ MSE ก่อนการเทรน: {mse:.6f}")
improvement = ((mse - final_mse) / mse) * 100
print(f"การเทรนช่วยลด MSE ลง: {improvement:.2f}%")

# ==================== ส่วนที่ 5: สรุปและบทบาทของ y ====================
print("\n" + "=" * 60)
print("ส่วนที่ 5: สรุปและบทบาทของ y")
print("=" * 60)

print("1. สรุปการเปลี่ยนแปลงของ Parameters:")
print(f"  W: {w_ih.item():.6f} → {rnn_train.weight_ih.item():.6f}")
print(f"  U: {w_hh.item():.6f} → {rnn_train.weight_hh.item():.6f}")
print(f"  V: {v_weight:.6f} → {output_train.weight.item():.6f}")

print("\n2. บทบาทของ y ในการเทรน:")
print("   - y ถูกคำนวณจาก hidden state (h) ในแต่ละ time step")
print("   - การเปรียบเทียบระหว่าง y กับค่าเป้าหมายทำให้เราสามารถคำนวณ loss")
print("   - loss ใช้ในการคำนวณ gradient เพื่อปรับปรุง parameters (W, U, V)")
print("   - หลังการเทรน parameters ถูกปรับให้ค่า y ใกล้เคียงกับเป้าหมายมากขึ้น")
print("   - ถ้าไม่มี y เราจะไม่สามารถเทรนโมเดลได้ เพราะไม่มีตัวเชื่อมระหว่างโมเดลกับข้อมูลจริง")

print("\n3. ประโยชน์ของ y ในการทำนาย load forecasting:")
print("   - ในกรณี load forecasting, y คือปริมาณการใช้ไฟฟ้าที่ทำนาย")
print("   - input (x) คือข้อมูลในอดีต เช่น การใช้ไฟฟ้าในชั่วโมงก่อนหน้า")
print("   - RNN เรียนรู้ความสัมพันธ์ระหว่าง x กับ y เพื่อทำนายค่าการใช้ไฟฟ้าในอนาคต")
print("   - ยิ่งค่า y ใกล้เคียงกับค่าจริงมากเท่าไร การวางแผนการผลิตไฟฟ้าก็จะแม่นยำมากขึ้นเท่านั้น")

ส่วนที่ 1: การเตรียมโมเดลและข้อมูล
Parameters เริ่มต้นของ RNN Cell:
Weight W (input-to-hidden): 0.3116
Weight U (hidden-to-hidden): -0.3960
Weight V (hidden-to-output): 0.5000

สร้างชุดข้อมูลผสมความยาว 20 ค่า (3 ค่าแรกคือ [1, -0.7775, 0.2588] ส่วนที่เหลือเป็น sine wave)
จำนวน time steps ในชุดข้อมูล: 19
Shape ของ input (x): torch.Size([19, 1]), target (y): torch.Size([19, 1])

ข้อมูลทั้งหมด 20 ค่า:
  ลำดับ |    ค่าข้อมูล    
-------------------------
    1   |    +1.000000    
    2   |    -0.777500    
    3   |    +0.258800    
    4   |    +0.915773    
    5   |    +0.475947    
    6   |    -0.164595    
    7   |    -0.735724    
    8   |    -0.996584    
    9   |    -0.837166    
   10   |    -0.324700    
   11   |    +0.324700    
   12   |    +0.837167    
   13   |    +0.996584    
   14   |    +0.735724    
   15   |    +0.164594    
   16   |    -0.475948    
   17   |    -0.915773    
   18   |    -0.969400    
   19   |    -0.614212    
   20   |    +0.000000    

ข้อมู